In [1]:
import graphlab
import requests

In [2]:
discogs_frame = graphlab.SFrame('discogs.csv')

This non-commercial license of GraphLab Create for academic use is assigned to jbmlaird@gmail.com and will expire on April 20, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\j\AppData\Local\Temp\graphlab_server_1493833811.log.0


Finished parsing file G:\Git\DiscogsML\discogs.csv

Parsing completed. Parsed 100 lines in 0.817832 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Read 26986418 lines. Lines per second: 4.25706e+006

Read 50268723 lines. Lines per second: 4.39538e+006

Read 72792719 lines. Lines per second: 4.40789e+006

Finished parsing file G:\Git\DiscogsML\discogs.csv

Parsing completed. Parsed 90023677 lines in 19.9065 secs.

## Rating: 0 unrated, 1-5 rated

## Category: 0 = collection, 1 = wantlist, 2 = for sale

In [3]:
discogs_frame.rename({'1':'userId', '63257':'releaseId', '0':'rating','0.1':'category'}) # Lost a row here renaming. Add it later

userId,releaseId,rating,category
1,31435,0,0
1,77503,0,0
1,19282,0,0
1,71232,0,0
1,83485,0,0
1,32466,0,0
1,5471,5,0
1,5482,0,0
1,10372,0,0
1,13148,0,1


In [4]:
graphlab.canvas.set_target('ipynb')
discogs_frame.show()

#### ~215,000  users

# Find the most occuring releases

In [5]:
most_frequent_releases = discogs_frame.groupby(key_columns={'releaseId'}, operations={'appearance_count':
                                                           graphlab.aggregate.COUNT('releaseId')}).sort('appearance_count', ascending=False)

In [6]:
most_frequent_releases

releaseId,appearance_count
4570366,9825
526351,8443
367104,7276
2911293,7031
1587168,6403
5764967,6159
74260,6127
194021,6046
1343227,5748
242785,5455


## Get the human readable names of these releases

In [7]:
discogskey = open('discogskey.txt').readline() # Get my Discogs API token

In [8]:
payload = {'token':discogskey}
def get_release_name(releaseId):
    response = requests.get('https://api.discogs.com/releases/' + str(releaseId), params=payload)
    print response.json()['artists'][0]['name'] + " " + response.json()['title']

In [9]:
for releaseId in most_frequent_releases['releaseId'][0:10]:
    get_release_name(str(releaseId))

Daft Punk Random Access Memories
Fleetwood Mac Rumours
Pink Floyd The Dark Side Of The Moon
Michael Jackson Thriller
Radiohead OK Computer
Jack White (2) Lazaretto
Godspeed You Black Emperor! Lift Your Skinny Fists Like Antennas To Heaven
Prince And The Revolution Purple Rain
Bruce Springsteen Born In The U.S.A.


## Number of rated releases

In [10]:
print "One star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==1]), ",d"))
print "Two star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==2]), ",d"))
print "Three star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==3]), ",d"))
print "Four star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==4]), ",d"))
print "Five star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==5]), ",d"))

One star ratings: 133,695
Two star ratings: 342,173
Three star ratings: 1,537,875
Four star ratings: 3,138,727
Five star ratings: 4,486,020


In [11]:
print "Unrated: " + str(format(len(discogs_frame[discogs_frame['rating']==0]), ",d"))

Unrated: 80,385,187


## Create a subset containing only collection/wantlist/for sale without the 3 star ratings or below

In [4]:
no_star_frame = discogs_frame[discogs_frame['rating']==0]
four_or_five_star_frame = discogs_frame[discogs_frame['rating']>3]

In [13]:
no_star_frame.show()

In [5]:
no_four_five_star_frame = no_star_frame.append(four_or_five_star_frame)

In [15]:
no_four_five_star_frame.show()

## Create an item similarity recommender

In [6]:
train_data, test_data = graphlab.recommender.util.random_split_by_user(no_four_five_star_frame,
                                                                      user_id='userId',
                                                                      item_id='releaseId',
                                                                      max_num_users=None,
                                                                      random_seed=0)

In [9]:
print len(train_data)
print len(test_data)

70407893
17602041


In [ ]:
item_similarity_model = graphlab.recommender.item_similarity_recommender.create(train_data, 
                                                                   user_id='userId',
                                                                   item_id='releaseId',
                                                                   # don't use target= as '0' ratings exist
                                                                   similarity_type='jaccard', #cosine or pearson
                                                                   )

Recsys training: model = item_similarity

Warning: Ignoring columns rating, category;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

+------------------------------------+------------+

| 9.57s                              | 0          |

| 10.52s                             | 20.75      |

| 11.83s                             | 37.75      |

| 12.52s                             | 63.25      |